In [ ]:
from functools import total_ordering
import glob
import os
import matplotlib.pyplot as plt
from PIL import Image
import math
from pathlib import Path
import shutil
import torch
import os
import cv2
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import jaccard_score

In [ ]:
predictions_directory = "/home/wej36how/codes/CoSOD-main/result/Predictions/NWRDFRust"
groundTruths_directory = "/home/wej36how/datasets/NWRDF/test/masks"
save_dir = main_directory+"_concatenated"
patch_size = (224, 224)

Copies all the subdirectories containing parts in their name to their corresponding main image direcotry.

In [ ]:
def organize_subdir(main_dir):
    subdirs = [d for d in main_dir.iterdir() if d.is_dir()]
    for subdir in subdirs:
        parts_dirs = main_dir.glob(f'{subdir.name}_*')
        for part_dir in parts_dirs:
            for file in part_dir.iterdir():
                if file.is_file():
                    shutil.move(str(file), str(subdir))
            shutil.rmtree(part_dir)
            
organing_subdir(prediction_directory)

Concatenate The patches to form the final prediction Map and store it in save_dir

In [ ]:
def get_patch_position(img_shape, patch_size, patch_no):
    num_patches_horizontally = img_shape[1] // patch_size[1]
    num_patches_vertically = img_shape[0] // patch_size[0]
    
    if patch_no >= num_patches_horizontally * num_patches_vertically:
        raise ValueError("Invalid patch number for the given image size and patch size.")
    
    row = patch_no // num_patches_horizontally
    col = patch_no % num_patches_horizontally
    
    return row, col

def replace_with_patch(matrix, patch, row, col, patch_size):
    start_y = row * patch_size[0]
    start_x = col * patch_size[1]

    end_y = start_y + patch_size[0]
    end_x = start_x + patch_size[1]

    end_y = min(end_y, matrix.shape[0])
    end_x = min(end_x, matrix.shape[1])

    patch = patch[:end_y - start_y, :end_x - start_x]
    matrix[start_y:end_y, start_x:end_x] = patch
    
    return matrix

def concatenate_patches():
    masks = glob.glob(main_directory+'/*')
    for mask in masks:
        patches = glob.glob(f"{mask}/*")
        imgNo = mask.split('/')[-1]
        orig_mask_path = ground_truth_masks_directory+f"/{imgNo}.png"
        img = cv2.imread(orig_mask_path, 0)
        pred = np.zeros(img.shape)
        for patch_path in patches:
            patchNo = patch_path.split('/')[-1][:-4].split('.')[0].split('_')[-1]
            row, col = get_patch_position(img.shape, patch_size, int(patchNo))
            patch = cv2.imread(patch_path, 0)
            patch = cv2.resize(patch, (224, 224), interpolation=cv2.INTER_AREA)
            pred = replace_with_patch(pred, patch, row, col, patch_size)
        
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        cv2.imwrite(os.path.join(save_dir, f"{imgNo}.png"), pred)

concatenate_patches()


Calculate Precision, Recall, F1, IOU for rust class

In [ ]:
def Evaluate(groundTruths_directory, predictions_directory):
    precisions = []
    recalls = []
    f1_scores = []
    iou_scores = []

    for filename in os.listdir(predictions_directory):
        pred_path = os.path.join(predictions_directory, filename)
        gt_path = os.path.join(groundTruths_directory, filename)
       
        if os.path.exists(pred_path) and os.path.exists(gt_path):
            pred_img = cv2.imread(pred_path, cv2.IMREAD_GRAYSCALE)
            gt_img = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)
            
            pred_flat = pred_img.flatten()
            gt_flat = gt_img.flatten()
            
            pred_flat = (pred_flat > 127).astype(np.uint8)
            gt_flat = (gt_flat > 127).astype(np.uint8)
            
            # Default average argument = binary (Only report results for the class specified by pos_label) 
            # Default pos_label = 1 so metrics are calculated for rust class only
            precision = precision_score(gt_flat, pred_flat)
            recall = recall_score(gt_flat, pred_flat)
            f1 = f1_score(gt_flat, pred_flat)
            iou = jaccard_score(gt_flat, pred_flat)
            
            precisions.append(precision)
            recalls.append(recall)
            f1_scores.append(f1)
            iou_scores.append(iou)

    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)
    avg_f1_score = np.mean(f1_scores)
    avg_iou = np.mean(iou_scores)

    print(f'Average Precision: {avg_precision:.4f}')
    print(f'Average Recall: {avg_recall:.4f}')
    print(f'Average F1 Score: {avg_f1_score:.4f}')
    print(f'Average iou Score: {avg_iou:.4f}')

Evaluate(groundTruths_directory, predictions_directory)